## Running ISR from a notebook - testing CTI correction

Craig Lage - 28-Sep-22

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/main', collections=["LSSTCam/raw/all","LSSTCam/calib", \
                                           "u/cslage/sdf/BOT/cti_20220916"])

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False

isrConfig.doDeferredCharge=False

# Adjust these as needed and add as many more as you want

In [ ]:
expId = 3021120700215
exp1 = butler.get('raw', detector=74, exposure=expId)
exp2 = butler.get('raw', detector=74, exposure=expId)
exp3 = butler.get('raw', detector=74, exposure=expId)
biasExp = butler.get('bias', detector=74, exposure=expId)
deferredChargeCalib = butler.get('cpCtiCalib', detector=74, exposure=expId)
camera = butler.get('camera', instrument='LSSTCam')

In [ ]:
isrConfig.doDeferredCharge=False
isrTask1 = IsrTask(config=isrConfig)
isrResult1 = isrTask1.run(exp1, bias=biasExp) # No CTI correction
isrConfig.doDeferredCharge=True
isrTask2 = IsrTask(config=isrConfig)
isrResult2 = isrTask2.run(exp2, bias=biasExp, deferredCharge=deferredChargeCalib) # With CTI correction
isrConfig.doDeferredCharge=True
isrTask3 = IsrTask(config=isrConfig)
dcDict = deferredChargeCalib.toDict()
dcDict['driftScale']['C11'] = 1E-6
#dcDict['decayTime']['C17'] *= 1E-6
#dcDict['serialTraps']['C17']['emissionTime'] *= 1E-6
deferredChargeCalib.fromDict(dcDict)
isrResult3 = isrTask3.run(exp3, bias=biasExp, deferredCharge=deferredChargeCalib) # With CTI correction, driftScale=0

In [ ]:
for amp in camera[74].getAmplifiers():
    if amp.getName() not in ['C11']:
        continue
    notCtiCorrected = isrResult1.exposure.image[amp.getBBox()].array
    ctiCorrectedZero = isrResult3.exposure.image[amp.getBBox()].array
diff = ctiCorrectedZero - notCtiCorrected

In [ ]:
diff.max()

In [ ]:
diff.shape

In [ ]:
diff[1995:2000, 500:509]

In [ ]:
plt.imshow(diff[1995:2000, 500:511], interpolation='nearest')
plt.colorbar()

In [ ]:
y = 1000
plt.figure(figsize=(8,8))
plt.suptitle(f"CTI correction results {expId}")
plt.subplots_adjust(wspace=0.5)
xplot = range(460, 509)           
for amp in camera[74].getAmplifiers():
    if amp.getName() not in ['C11']:
        continue
    notCtiCorrected = isrResult1.exposure.image[amp.getBBox()].array
    ctiCorrected = isrResult2.exposure.image[amp.getBBox()].array
    ctiCorrectedZero = isrResult3.exposure.image[amp.getBBox()].array
    plt.subplot(2,2,1)
    plt.title(amp.getName())
    plt.plot(xplot, notCtiCorrected[y,460:509], label="notCtiCorrected")
    plt.plot(xplot, ctiCorrected[y,460:509], label="ctiCorrected")
    plt.ylim(65000, 72000)
    plt.legend()
    plt.subplot(2,2,2)
    plt.title(amp.getName())
    plt.plot(xplot, notCtiCorrected[y,460:509], label="notCtiCorrected")
    plt.plot(xplot, ctiCorrectedZero[y,460:509], label="ctiCorrected-driftScale=0")
    plt.ylim(65000, 72000)
    plt.legend()
    plt.subplot(2,2,3)
    plt.title(amp.getName())
    plt.plot(xplot, ctiCorrected[y,460:509] - notCtiCorrected[y,460:509], label="diff")
    #plt.ylim(65000, 72000)
    plt.legend()
    plt.subplot(2,2,4)
    plt.title(amp.getName())
    plt.plot(xplot, ctiCorrectedZero[y,460:509] - notCtiCorrected[y,460:509], label="diff - driftScale = 0")
    #plt.ylim(65000, 72000)
    plt.legend()
plt.savefig(f"/sdf/group/rubin/u/cslage/BOT_LSSTCam/cti/CTI_Test_{expId}_C11_03Oct22.pdf")

In [ ]:
deferredChargeCalib.toDict()

In [ ]:
print(deferredChargeCalib.decayTime['C17'])

In [ ]:
print(deferredChargeCalib.driftScale['C16'])

In [ ]:
print(deferredChargeCalib.decayTime['C17'])
print(deferredChargeCalib.serialTraps['C17'].emission_time)
dcDict = deferredChargeCalib.toDict()
dcDict['decayTime']['C17'] *=1E-6
dcDict['serialTraps']['C17']['emissionTime'] *= 1E-6
deferredChargeCalib.fromDict(dcDict)
print(deferredChargeCalib.decayTime['C17'])
print(deferredChargeCalib.serialTraps['C17'].emission_time)

In [ ]:
deferredChargeCalib.serialTraps['C17'].emission_time

In [ ]:
dcDict = deferredChargeCalib.toDict()

In [ ]:
dcDict['serialTraps']['C17']['emissionTime']

In [ ]:
dcDict['globalCti']['C17']

In [ ]:
dcTable = deferredChargeCalib.toTable()

In [ ]:
dcTable

In [ ]:
dcTable.